In [1]:
import numpy as np
import random

points=np.loadtxt('point_cloud')
points[:,(0,1,2)]=points[:,(1,2,0)]
points[:,2]=-(points[:,2]-np.mean(points[:,2]))+np.mean(points[:,2])
points[:,0]=2*np.mean(points[:,0])-points[:,0]

In [2]:
xs=[]
colors=[]
for id in np.unique(points[:,3]):
    xs.append(points[points[:,3]==id,0:3])
    colors.append((random.randint(0,255),random.randint(0,255),random.randint(0,255)))

colors=[(0,255,0),(0,255,255),(125,0,125),(255,255,255),(125,0,255)]
def draw_2D_points(frame,xs2d,c,center2d):
    xs2d[:,1]=-xs2d[:,1]
    xs2d+=center2d
    xs2d=xs2d[xs2d[:,0]>0,:]
    xs2d=xs2d[xs2d[:,1]>0,:]
    xs2d=xs2d[xs2d[:,0]<frame.shape[1]-1,:]
    xs2d=xs2d[xs2d[:,1]<frame.shape[0]-1,:]
    xs2d=np.uint16(xs2d)
    if len(xs2d)>0:
        frame[xs2d[:,1],xs2d[:,0],:]=c
    
def rotation_matrix(ths):
    return np.array([[np.cos(ths[2]),-np.sin(ths[2]),0,0],[np.sin(ths[2]),np.cos(ths[2]),0,0],[0,0,1,0],[0,0,0,1]]) \
    @ np.array([[np.cos(ths[1]),0,np.sin(ths[1]),0],[0,1,0,0],[-np.sin(ths[1]),0,np.cos(ths[1]),0],[0,0,0,1]])\
    @ np.array([[1,0,0,0],[0,np.cos(ths[0]),-np.sin(ths[0]),0],[0,np.sin(ths[0]),np.cos(ths[0]),0],[0,0,0,1]])
def translation_matrix(Ts):
    return np.array([[1,0,0,Ts[0]],[0,1,0,Ts[1]],[0,0,1,Ts[2]],[0,0,0,1]])
def draw_3D_points(frame, pts, c, ths, Ts, scale,center2d):
    camera_matrix = np.array([[scale , 0, 0, 0],[0, scale, 0, 0],[0, 0, 1, 0]]) @ rotation_matrix(ths) @ translation_matrix(Ts)
    hpts = np.concatenate((pts,np.ones((pts.shape[0],1))),axis=1).transpose()
    hpoints_2d=camera_matrix @ hpts
    xs2d=(hpoints_2d[0:2,:]/hpoints_2d[2,:]).transpose()
    draw_2D_points(frame,xs2d,c,center2d)
    return xs2d, camera_matrix, hpts

In [4]:
import cv2
import scipy.io as sio
import os.path

window_name='point cloud'


if os.path.isfile('point_parameters.mat'):
    parameters=sio.loadmat('point_parameters.mat')
    ths=parameters['ths'][0]
    Ts=parameters['Ts'][0]
    scale=parameters['scale'][0][0]
    center2d=parameters['center2d'][0]
else:
    ths=[0.,0.,0.]
    Ts=[420.,80.,-40.]
    scale = 100
    center2d=[1000,500]
    
Tstep=2.
thstep=0.005
scalestep=1.02
centerstep=10

while (True):
#     ret, frame = cap.read()
    frame = np.zeros((1080,1920,3),dtype=np.uint8)
    
#     draw_2D_points(frame,range(1,100),range(1,100),colors[0])
    for c, pts in zip(colors,xs):
        draw_3D_points(frame, pts, c, ths, Ts, scale, center2d)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,'T = (%1.1f, %1.1f, %1.1f)' % (Ts[0],Ts[1],Ts[2]), (10,20), font, 0.5,(255,255,255),1,cv2.LINE_AA)
    dths=np.array(ths)*180/np.pi
    cv2.putText(frame,'theta = (%1.1f, %1.1f, %1.1f)' % (dths[0],dths[1],dths[2]), (10,40), font, 0.5,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(frame,'center2d = (%d, %d)' % (center2d[0],center2d[1]), (10,60), font, 0.5,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(frame,'scale = %1.2f' % scale, (10,80), font, 0.5,(255,255,255),1,cv2.LINE_AA)
    cv2.imshow(window_name,frame)
#     out.write(frame)
    pressedKey = cv2.waitKeyEx(20) 
    if pressedKey & 0xFF == ord('q'):
        break
    elif pressedKey & 0xFF == ord('a'):
        Ts[0]+=Tstep
    elif pressedKey & 0xFF == ord('d'):
        Ts[0]-=Tstep
    elif pressedKey & 0xFF == ord('w'):
        Ts[1]+=Tstep
    elif pressedKey & 0xFF == ord('s'):
        Ts[1]-=Tstep
    elif pressedKey & 0xFF == ord('r'):
        Ts[2]+=Tstep
    elif pressedKey & 0xFF == ord('f'):
        Ts[2]-=Tstep
    elif pressedKey & 0xFF == ord('i'):
        ths[0]+=thstep
    elif pressedKey & 0xFF == ord('k'):
        ths[0]-=thstep
    elif pressedKey & 0xFF == ord('j'):
        ths[1]+=thstep
    elif pressedKey & 0xFF == ord('l'):
        ths[1]-=thstep
    elif pressedKey & 0xFF == ord('y'):
        ths[2]+=thstep
    elif pressedKey & 0xFF == ord('h'):
        ths[2]-=thstep
    elif pressedKey & 0xFF == ord('g'):
        center2d[1]-=centerstep
    elif pressedKey & 0xFF == ord('b'):
        center2d[1]+=centerstep
    elif pressedKey & 0xFF == ord('v'):
        center2d[0]-=centerstep
    elif pressedKey & 0xFF == ord('n'):
        center2d[0]+=centerstep
    elif pressedKey & 0xFF == ord('z'):
        scale *= scalestep
    elif pressedKey & 0xFF == ord('x'):
        scale /= scalestep
#     print(pressedKey)

#     print('w is pressed')
#     if cv2.waitKey(1) & 0xFF == ord('q'): # btw, you need to click the screen first. And then 
#                                          # press q to quit
#         break

sio.savemat('point_parameters.mat',{'scale':scale,'Ts':Ts,'ths':ths,'center2d':center2d})
# cap.release()
# out.release()
cv2.destroyAllWindows()

In [4]:
len(colors)

5